#### Import

In [57]:
from fastai.vision.widgets import *
from fastai.vision.all import *

#### Load

In [58]:
path = Path()
path.ls(file_exts='.pkl')
learn_inf = load_learner(path/'export.pkl')
learn_inf

## Deploy (Fastai version)

In [59]:
# Prepare buttons
btn_upload = widgets.FileUpload()
btn_run = widgets.Button(description="Classify")
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [60]:
# button for predicting using model
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl:
        display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f"Prediction: {pred}; Probability: {probs[pred_idx]:.04f}"

btn_run.on_click(on_click_classify)

In [61]:
# show widgets
VBox([widgets.Label("Select your digit! (will classify last uploaded)"), 
      btn_upload, btn_run, out_pl, lbl_pred])

## Deploy (from scratch ver)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange

import requests, gzip
from pathlib import Path

def fetch(url):    
    name = url.split("/")[-1]
    dirs = Path("dataset/mnist")
    path = (dirs / name)
    if path.exists():
        with path.open("rb") as f:
            dat = f.read()
    else:
        if not dirs.is_dir():
            dirs.mkdir(parents=True, exist_ok=True)
        with path.open("wb") as f:
            dat = requests.get(url).content
            f.write(dat)
    return np.frombuffer(gzip.decompress(dat), dtype=np.uint8).copy()

def mnist_dataset():
    X_train = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28, 28))
    Y_train = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")[8:]
    X_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28, 28))
    Y_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz")[8:]
    return (X_train, Y_train, X_test, Y_test)
X_train, Y_train, X_test, Y_test = mnist_dataset()

def _uniform(a, b, dtype=np.float32):
    return np.random.uniform(-1., 1., size=(a, b)).astype(dtype) / np.sqrt(a*b)

def linear(a, b):
    return a @ b # np.dot(a, b)

def relu(x): 
    return np.maximum(x, 0)

def logsumexp(x):
    c = x.max(axis=1)
    return x - (c + np.log(np.exp(x-c.reshape((-1, 1))).sum(axis=1))).reshape((-1, 1))

class Net:
    def __init__(self, lr=0.001):
        # parameters
        self.w1 = _uniform(784, 128)
        self.w2 = _uniform(128, 10)
        self.lr = lr
    
    def calc_loss(self):
        # crossentropyloss
        self.x_lsm = logsumexp(self.x_w2)
        self.loss = (-self.out * self.x_lsm).mean(axis=1).mean()
    
    def calc_optim(self):
        # Update (SGD)
        self.w1 += -self.lr * self.d_w1
        self.w2 += -self.lr * self.d_w2
    
    def forward(self, X):
        self.X = X
        self.x_w1 = linear(X, self.w1)
        self.x_relu = relu(self.x_w1)
        self.x_w2 = linear(self.x_relu, self.w2)
        return self.x_w2
    
    def backward(self, Y):
        # target -> one-hot encoded
        self.out = np.zeros((len(Y),10), np.float32)
        self.out[range(self.out.shape[0]), Y] = 1
        # loss function
        self.calc_loss()
        # derivative of target
        d_out = -self.out / len(Y)
        # derivative of loss with respect to target
        dx_lsm = d_out - np.exp(self.x_lsm) * d_out.sum(axis=1).reshape((-1, 1))
        # derivative of l2
        self.d_w2 = self.x_relu.T @ dx_lsm
        # derivative of ReLU
        d_relu = dx_lsm @ self.w2.T
        d_relu[self.x_w1 < 0] = 0
        # derivative of l1
        self.d_w1 = self.X.T @ d_relu
        

net = Net()
batch_size = 128
losses, accuracies = [], []
## Train
for i in (t := trange(10000)):
    # Batch of training data & target data
    samp = np.random.randint(0, X_train.shape[0], size=(batch_size))
    X = X_train[samp].reshape((-1, 28*28))
    Y = Y_train[samp]
    
    output = net.forward(X)
    net.backward(Y)
    net.calc_optim()
    
    # Save for statistic
    cat = np.argmax(output, axis=1) # results from Net
    acc = (cat == Y).mean()
    accuracies.append(acc)
    losses.append(net.loss)
    
    t.set_description(f"Loss: {net.loss:.5f}; Acc: {acc:.5f}")
    
#plt.ylim(-0.01, 1.1)
#plt.plot(losses)
#plt.plot(accuracies)
#plt.legend(["losses", "accuracies"])
#plt.show()

## Evaluation
Y_test_preds = np.argmax(net.forward(X_test.reshape((-1, 28*28))), axis=1)
true_acc = (Y_test == Y_test_preds).mean()
print(f"Accuracy on testing set: {true_acc}")

Loss: 0.00153; Acc: 0.99219:  94%|█████████▍| 9444/10000 [00:37<00:02, 265.99it/s]

In [ ]:
from ipywidgets import VBox, FileUpload, Button, Output, Label
from IPython.display import display
from PIL import Image, ImageOps
import io

def softmax(x): return np.exp(x) / np.exp(x).sum(axis=1)

btn_upload_ = FileUpload()
btn_run_ = Button(description="Classify")
out_pl_ = Output()
lbl_pred_ = Label()

def on_click_classify_(change):
    # Convert Image from bytes to PIL format and than to numpy array
    fn = io.BytesIO(btn_upload_.data[-1])
    img = Image.open(fn).resize((28, 28))
    img_ = ImageOps.grayscale(img)
    arr = np.asarray(img_)
    
    # get values from NN and show them
    output = net.forward(arr.reshape((1, 28*28)))
    predict = np.argmax(output, axis=1)
    confid = softmax(output)
    
    # display
    out_pl_.clear_output()
    with out_pl_: 
        display(img_)
        lbl_pred_.value = f"Prediction: {predict[0]}; Confidence: {confid[0][predict[0]]:.4f}"
    
btn_run_.on_click(on_click_classify_)
VBox([Label("Select your digit! (will classify last uploaded)"), 
      btn_upload_, btn_run_, out_pl_, lbl_pred_])

### @ludius0 testing deployment from jupyter notebook to web (voila)